In [30]:
import pandas as pd
import numpy as np

import snscrape.modules.twitter as sntwitter

import nltk
from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

import collections
from argparse import Namespace
from functools import partial
from pathlib import Path
from pprint import pprint

import os
import pickle
import random
import re
import string


In [31]:
df = pd.read_csv("https://raw.githubusercontent.com/lifeonthefence/text-success/main/Data/Ukraine%20Tweets.csv")

In [32]:
df.insert(loc=6,
         column="Adjusted Tweet",
         value=df["rendered_content"])



In [33]:
import re

def remove_mentions_hashtags(text):
    text = re.sub("#[A-Za-z0-9_]+","", text)
    return text

df["Adjusted Tweet"] = df["Adjusted Tweet"].apply(remove_mentions_hashtags)

In [34]:
def remove_links(text):
    text = re.sub('http://\S+|https://\S+', '', text)
    text = re.sub('http[s]?://\S+', '', text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'bit.ly/\S+', '', text) # remove bitly links
    text = text.strip('[link]') # remove [links]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_links)

In [35]:
! pip install emot
! pip install emoji

import emot
import emoji

def demote(text):
    text = emoji.demojize(text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(demote)

In [36]:
df.head()

,id,date,user,user_followers,user_created,rendered_content,Adjusted Tweet,likes,retweets,replies,quoteCount,hashtags,lang,media,mentionedUsers
0,1477420789863436289,2022-01-01 23:25:40+00:00,anno1540,8838,2014-06-12 17:05:22+00:00,"Lithuania will never abandon Ukraine, voluntee...","Lithuania will never abandon Ukraine, voluntee...",5,1,0,0,"['Lithuania', 'Ukraine']",en,NaN,NaN
1,1477414596424220679,2022-01-01 23:01:03+00:00,weather_odessa,119,2019-07-10 08:34:22+00:00,#odessa #odesa #ukraine #одесса\nNow: 4.2°C\nT...,#одесса\nNow: 4.2°C\nToday's Min: 4.2°C at ...,0,0,0,0,"['odessa', 'odesa', 'ukraine', 'одесса']",en,NaN,NaN
2,1477414332376010752,2022-01-01 23:00:00+00:00,AlArabiya_Eng,927174,2009-02-28 08:31:32+00:00,After tough talk between Presidents Joe Biden ...,After tough talk between Presidents Joe Biden ...,4,0,3,0,"['Russia', 'Ukraine']",en,NaN,NaN
3,1477409748572151809,2022-01-01 22:41:47+00:00,beatravelling,6329,2014-02-28 21:25:33+00:00,The beach can be nice in the fall too 😊🇺🇦\n\n#...,The beach can be nice in the fall too :smiling...,0,0,0,0,"['lanzheron', 'langeron', 'beach', 'odessa', '...",en,NaN,NaN
4,1477409332820119552,2022-01-01 22:40:08+00:00,TornCurtain1991,677,2012-02-08 15:30:41+00:00,"A note: Stepan #Bandera, DOB 01011909, was lea...","A note: Stepan , DOB 01011909, was leader of O...",1,2,0,0,"['Bandera', 'Ukraine']",en,NaN,NaN


In [37]:
df.set_index('id').to_dict()['Adjusted Tweet']

{1477420789863436289: 'Lithuania will never abandon Ukraine, volunteer helping Ukrainian army says\n\n \neuromaidanpress.com/2021/12/29/lit…',
 1477414596424220679: "   #одесса\nNow: 4.2°C\nToday's Min: 4.2°C at 00:28:52\nToday's Max: 4.4°C at 00:00:00\nMonth's Min: 2.9°C at 2022-01-01 00:00:00\nMonth's Max: 6.9°C at 2022-01-01 11:35:31",
 1477414332376010752: "After tough talk between Presidents Joe Biden and Vladimir Putin over 's troop buildup on the  border, both sides insist they are hopeful that a pathway to easing tensions could open during diplomatic talks set for January.\n\nenglish.alarabiya.net/News/world/202…",
 1477409748572151809: 'The beach can be nice in the fall too :smiling_face_with_smiling_eyes::Ukraine:\n\n    \n\nbeatravelling.com/2022/01/01/lan…',
 1477409332820119552: 'A note: Stepan , DOB 01011909, was leader of Organization of Ukrainian Nationalists (OUN) whose members civilian &amp; military in Ukrainian Insurgent Army (UPA) fought vs invaders of : Russian, N

In [38]:
dictionary_content = df.set_index("Adjusted Tweet")

In [39]:
pip install py-readability-metrics

Note: you may need to restart the kernel to use updated packages.


In [40]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zofiachoinska/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [41]:
nltk.download('punkt')

SENT_DETECTOR = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zofiachoinska/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
! python -m nltk.downloader punkt

/Users/zofiachoinska/opt/anaconda3/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zofiachoinska/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [43]:
pip install NLP-python

Note: you may need to restart the kernel to use updated packages.


In [44]:
def lowercase(text):    
    text = text.lower()
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(lowercase)

In [45]:
import string

def punctuation_remove(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(punctuation_remove)

In [46]:
from nltk import word_tokenize

def tokenize(text):
    text = word_tokenize(text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(tokenize)

In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def stem(text):
    stemmed = [porter.stem(word) for word in text]
    return text
df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(stem)

In [ ]:
my_list = df['Adjusted Tweet'].tolist()

In [ ]:
df['Adjusted Tweet'].tolist()

In [ ]:
combine_list = [element for nestedlist in my_list for element in nestedlist]

In [ ]:
combine_list

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tr_idf_model  = TfidfVectorizer()
tf_idf_vector = tr_idf_model.fit_transform(combine_list)

In [ ]:
print(type(tf_idf_vector), tf_idf_vector.shape)

In [ ]:
tf_idf_array = tf_idf_vector.toarray()
 
print(tf_idf_array)

In [48]:
df_tf_idf = pd.DataFrame(tf_idf_array, columns = combine_list)
 
df_tf_idf

ValueError: Shape of passed values is (1220537, 72862), indices imply (1220537, 1220537)

# test_corpus with one tweet

In [102]:
test_corpus = ['lithuania', 'will', 'never', 'abandon', 'ukraine', 'volunteer', 'helping', 'ukrainian', 'army', 'says', 'euromaidanpresscom20211229lit…']

In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [104]:
tr_idf_model  = TfidfVectorizer()
tf_idf_vector = tr_idf_model.fit_transform(test_corpus)

In [105]:
print(type(tf_idf_vector), tf_idf_vector.shape)

<class 'scipy.sparse._csr.csr_matrix'> (11, 11)


In [106]:
tf_idf_array = tf_idf_vector.toarray()
 
print(tf_idf_array)

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [108]:
words_set = tr_idf_model.get_feature_names()
 
print(test_corpus)

['lithuania', 'will', 'never', 'abandon', 'ukraine', 'volunteer', 'helping', 'ukrainian', 'army', 'says', 'euromaidanpresscom20211229lit…']


In [110]:
df_tf_idf = pd.DataFrame(tf_idf_array, columns = test_corpus)
 
df_tf_idf

,lithuania,will,never,abandon,ukraine,volunteer,helping,ukrainian,army,says,euromaidanpresscom20211229lit…
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
